In [1]:
using Pkg; Pkg.activate("C:\\Users\\KarlH\\Desktop\\ComputationalPhysicsMaster\\Code\\OrbitalNeuralMethods")
using Revise
using OrbitalNeuralMethods

  Activating project at `C:\Users\KarlH\Desktop\ComputationalPhysicsMaster\Code\OrbitalNeuralMethods`
┌ Info: Precompiling OrbitalNeuralMethods [6de8678a-dd1c-461e-a987-24da1d9562b4]
└ @ Base loading.jl:1423


In [2]:
import LinearAlgebra as la

In [3]:
using BenchmarkTools: @btime

In [4]:
function pprint(tensor)
    display(round.(tensor[n+1:l, n+1:l, 1:n, 1:n], digits = 5)[:, :, 2, 1])
end

pprint (generic function with 1 method)

In [5]:
function uprint(tensor)
    display(round.(tensor, digits = 5)[:, :, 2, 1])
end

uprint (generic function with 1 method)

## HO Basis and Spatial System

In [33]:
l = 5
ω = 0.25
basis = SpinBasis(HOBasis(l, ω))

n = 2
V = ShieldedCoulomb(0.25)
grid = [x for x in range(-10, stop = 10, length = 2001)]
@time system = System(n, basis, grid, V);
l = 2 * l

  0.031511 seconds (362 allocations: 1.323 MiB)


10

## CCSD

In [209]:
ccsd = setup_CCSD(system);
energy(ccsd)

0.644720024887512

In [224]:
CCSD_Update!(ccsd);
energy(ccsd)

8×2 Matrix{Float64}:
  1.40949e-12   0.0
  0.0           1.40949e-12
 -6.05006e-11   0.0
  0.0          -6.05007e-11
  6.14431e-13   0.0
  0.0           6.14431e-13
 -2.99303e-11   0.0
  0.0          -2.99303e-11

0.8262688101039434

In [ ]:
array([[-0.     ,  0.     ],
       [ 0.     , -0.     ],
       [-0.35986,  0.     ],
       [ 0.     , -0.35986],
       [-0.     ,  0.     ],
       [ 0.     , -0.     ],
       [-0.17758,  0.     ],
       [ 0.     , -0.17758]])

In [52]:
pprint(ccsd.t2)

8×8 Matrix{Float64}:
 -0.0       0.6855   -0.0      -0.0      …   0.23397  -0.0      -0.0
 -0.6855   -0.0      -0.0      -0.0         -0.0       0.0      -0.0
 -0.0       0.0      -0.0       0.30334     -0.0      -0.0       0.13369
 -0.0      -0.0      -0.30334  -0.0         -0.0      -0.13369  -0.0
 -0.0       0.23397  -0.0      -0.0          0.15818  -0.0       0.0
 -0.23397  -0.0       0.0      -0.0      …  -0.0      -0.0      -0.0
 -0.0       0.0      -0.0       0.13369      0.0      -0.0       0.09357
 -0.0      -0.0      -0.13369  -0.0         -0.0      -0.09357  -0.0

In [104]:
err = zero(ccsd.t1)
for a in n+1:l
    for i in 1:n
        Ω_1 = 0
        for k in 1:n
            for l in 1:n
                for c in n+1:10
                    Ω_1 -= system.u[k, l, c, i] * ccsd.t1[c, k] * ccsd.t1[a, l]
                end
            end
        end
        err[a, i] = Ω_1
    end
end
round.(err, digits= 4)[n+1:end, :]

8×2 Matrix{Float64}:
  0.0      0.0
  0.0      0.0
 -0.1334   0.0
  0.0     -0.1334
  0.0      0.0
  0.0      0.0
 -0.0255   0.0
  0.0     -0.0255